In [28]:
import matplotlib.pyplot as plt
import random as r
import pprint as pp
import numpy as np
import csv
import tensorflow as tf
from tensorflow import keras

In [29]:
# Define type of slice and its parameters
slices_type = {
    'eMBB' : {
        'bandwidth_max': 100000000, #max bandwidth which BS can give to this slice
        'client_weight': 0.3, # Portion of clients which use this type of slice
        'band_guaranteed' : 0, # based on quality of service
        'min': 4000000, 
        'max': 500000000 # how much bps can require this type of slice
      },
      'URLLC' : {
        'bandwidth_max': 10000000,
        'client_weight': 0.2,
        'band_guaranteed' : 5000000,
        'min': 5000000,
        'max': 8000000
      },
      'MIoT' : {
        'bandwidth_max': 10000000,
        'client_weight': 0.15,
        'band_guaranteed' : 1000000,
        'min': 1000000, 
        'max': 8000000
      },
      'mMTC' : {
        'bandwidth_max': 10000000,
        'client_weight': 0.15,
        'band_guaranteed' : 1000000,
        'min': 1000000, 
        'max': 8000000
      },
      'voice' : {
        'bandwidth_max': 1000000,
        'client_weight': 0.2,
        'band_guaranteed' : 500000,
        'min': 4000000, 
        'max': 8000000
     }
}
          
# Define base station parameters
basestation = {
    'capacity': 20000000000,   #20Gbps
    'coverage': 250,
    'ratios': { # portion of capacity dedicated to various slice type
        'URLLC': 0.05,
        'eMBB': 0.55,
        'mMTC': 0.15,
        'voice': 0.1,
        'MIoT': 0.15
    },
    'x': 300,
    'y': 300
}

# Define mobility patterns and its parameters
mobility_patterns = {
  'car': {
    'distribution': 'normal',
    'params':(0,7),
    'client_weight': 0.10
  },
  'walk': {
    'distribution': 'randint',
    'params': (-1, 1),
    'client_weight': 0.40
  },
  'stationary': {
    'distribution': 'normal',
    'params': (0, 0.1),
    'client_weight': 0.20
  },
  'publictransport': {
    'distribution': 'randint',
    'params': (-4, 4),
    'client_weight': 0.10
  },
  'slackperson':{
    'distribution': 'randint',
    'params':(0, 1),
    'client_weight': 0.20
  }
}

In [30]:
# Define obj Client
class Client:
    def __init__(self, ID, x, y, mobility_pattern, freq, slice_type, bs, slice_cap = None):
        self.ID = ID
        self.x = x
        self.y = y
        self.mobility_pattern = mobility_pattern
        self.freq = freq
        self.slice_type = slice_type
        self.bs = bs
        self.usage_rem = 0
        self.last_usage = 0
        self.connected = False
    
    def step_1(self):
        if self.usage_rem > 0:
            if self.connected:
                self.start_consume()
            else:
                self.connect() # if connection refused, usage remaining in memory
        else:
            if self.connected:
                self.disconnect()
            else:
                self.generate_usage_and_connect()
        
    def step_2(self):
        if self.connected and self.last_usage > 0:
            self.release_consume()
            if self.usage_rem == 0:
                self.disconnect()
    
    def step_3(self):
        x, y = self.move()
        self.x += x
        self.y += y    
                
    def generate_usage_and_connect(self):
        if self.slice_type is not None:
            # Generate new usage
            self.usage_rem = self.generate()
            self.connect()
            #print(f"Client_{self.ID}, coordinates : ({self.x}, {self.y}) requests {self.usage_rem} usage to BS{self.bs.ID} to slice {self.slice_type}")     

    def connect(self):
        s = self.slice_type
        
        if self.connected:
            return
        
        if self.is_avaliable():         # threshold for usage
            self.connected = True
            
            if s == 'eMBB':
                bs.embb_users += 1
            if s == 'URLLC':
                bs.urllc_users += 1
            if s == 'MIoT':
                bs.miot_users += 1
            if s == 'mMTC':
                bs.mmtc_users += 1
            if s == 'voice':
                bs.voice_users += 1
            
            #print(f"Client {self.ID}, coordinates : ({self.x}, {self.y}) connected to slice {s} @ BS{self.bs.ID}")
        else:
            self.connected = False
            #print(f"Client {self.ID}, coordinates : ({self.x}, {self.y}) connection refused to slice {s} @ BS{self.bs.ID}")
        
    def disconnect(self):
        s = self.slice_type

        if s == 'eMBB':
            bs.embb_users -= 1
        if s == 'URLLC':
            bs.urllc_users -= 1
        if s == 'MIoT':
            bs.miot_users -= 1
        if s == 'mMTC':
            bs.mmtc_users -= 1
        if s == 'voice':
            bs.voice_users -= 1
                
        self.connected = False
        self.usage_rem = 0
        
        #print(f"Client {self.ID}, coordinates: ({self.x},{self.y}) disconnected from slice {self.slice_type} @ BS{self.bs.ID} ")
    
    def start_consume(self):
        s = self.slice_type
        amount = min(self.usage_rem, self.get_consumable_share())    # bandwidth allocation
        self.last_usage = amount
        if s == 'eMBB':
            bs.embb_active += 1
            bs.porcodio += amount
            bs.uno += self.usage_rem
        if s == 'URLLC':
            bs.urllc_active += 1
            bs.diocane += amount
            bs.due += self.usage_rem
        if s == 'MIoT':
            bs.miot_active += 1
            bs.dioboia += amount
            bs.tre += self.usage_rem
        if s == 'mMTC':
            bs.mmtc_active += 1
            bs.boiadio += amount
            bs.quattro += self.usage_rem
        if s == 'voice':
            bs.voice_active += 1
            bs.diobestia += amount
            bs.cinque += self.usage_rem
        #print(f"Client {self.ID}, coordinates: ({self.x}, {self.y}) gets {amount} usage from @ BS{self.bs.ID} for slice {self.slice_type}")
           
    def release_consume(self):
        # release resources
        if self.last_usage > 0:
            self.usage_rem -= self.last_usage
            #print(f"Client {self.ID}, coordinates: ({self.x}, {self.y}), releases {self.last_usage} usage to BS{self.bs.ID}")
            self.last_usage = 0
        
    # update client coordinates    
    def move(self):   
        distr = self.mobility_pattern.distribution
        params = self.mobility_pattern.params
        if distr == "randint":
            delta_x = r.randint(params[0], params[1])
            delta_y = r.randint(params[0], params[1])
            return delta_x, delta_y
        if distr == "normal":
            delta_x = np.random.normal(params[0], params[1])
            delta_y = np.random.normal(params[0], params[1])
            return delta_x, delta_y 
        
    # generate client consume   
    def generate(self):
        s = self.slice_type
        cicciolina = r.randint(slices_type[s]['min'], slices_type[s]['max'])
        return cicciolina
    
    def is_avaliable(self):
        global capacity
        s = self.slice_type
        
        if s == 'eMBB':
            bandwidth_next = capacity[s] / (bs.embb_users + 1)
        if s == 'URLLC':
            bandwidth_next = capacity[s] / (bs.urllc_users + 1)
        if s == 'MIoT':
            bandwidth_next = capacity[s] / (bs.miot_users + 1)
        if s == 'mMTC':
            bandwidth_next = capacity[s] / (bs.mmtc_users + 1)
        if s == 'voice':
            bandwidth_next = capacity[s] / (bs.voice_users + 1)
       
        if bandwidth_next < slices_type[s]['band_guaranteed']:
            return False

        return True
      
    def get_consumable_share(self):
        global capacity
        s = self.slice_type

        if s == 'eMBB':
            if bs.embb_users <= 0:
                return int(min(capacity[s], slices_type[s]['bandwidth_max']))
            else: 
                return int(min(capacity[s]/bs.embb_users, slices_type[s]['bandwidth_max']))
        
        if s == 'URLLC':
            if bs.urllc_users <= 0:
                return int(min(capacity[s], slices_type[s]['bandwidth_max']))
            else:
                return int(min(capacity[s]/bs.urllc_users, slices_type[s]['bandwidth_max']))
        
        if s == 'MIoT':
            if bs.miot_users <= 0:
                return int(min(capacity[s], slices_type[s]['bandwidth_max']))
            else:
                return int(min(capacity[s]/bs.miot_users, slices_type[s]['bandwidth_max']))
        
        if s == 'mMTC':
            if bs.mmtc_users <= 0:
                return int(min(capacity[s], slices_type[s]['bandwidth_max']))
            else: 
                return int(min(capacity[s]/bs.mmtc_users, slices_type[s]['bandwidth_max']))
        
        if s == 'voice':
            if bs.voice_users <= 0:
                return int(min(capacity[s], slices_type[s]['bandwidth_max']))
            else:
                return int(min(capacity[s]/bs.voice_users, slices_type[s]['bandwidth_max']))
        

In [31]:
# Define obj Mobility 
class Mobility:
    def __init__(self, name, distribution, params):
        self.name = name
        self.distribution = distribution
        self.params = params

# Define obj BaseStation
class BaseStation:
    def __init__(self,ID, x, y, coverage, capacity, ratios):
        self.ID =ID
        self.x = x
        self.y = y
        self.coverage = coverage
        self.capacity = capacity
        self.ratios = ratios
        
        self.embb_users = 0
        self.urllc_users = 0
        self.miot_users = 0
        self.mmtc_users = 0
        self.voice_users = 0
        self.embb_active = 0
        self.urllc_active = 0
        self.miot_active = 0
        self.mmtc_active = 0
        self.voice_active = 0
        self.porcodio = 0
        self.diocane = 0
        self.dioboia = 0
        self.boiadio = 0
        self.diobestia = 0
        self.uno = 0
        self.due = 0
        self.tre = 0
        self.quattro = 0
        self.cinque = 0
        
        

In [32]:
def get_random_mobility_pattern(vals, mobility_patterns):
    i = 0
    j = r.random()

    while vals[i] < j:
        i += 1

    return mobility_patterns[i]

def get_random_slice_type(vals):
    i = 0
    j = r.random()
    s = ''

    while vals[i] < j:
        i += 1
    
    if i == 0:
       s = 'eMBB'
    if i == 1:
       s = 'URLLC' 
    if i == 2:
       s = 'MIoT' 
    if i == 3:
       s = 'mMTC' 
    if i == 4:
       s = 'voice' 
    
    return s

def inRange(client, bs):
    x_client = client.x
    y_client = client.y
    x_bs = bs.x
    y_bs = bs.y
    if (abs(x_client - x_bs) < bs.coverage) and (abs(y_client - y_bs) < bs.coverage):
        return True
        print('true')
    else:
        return False
        print('false')

def get_new_ratios(prediction):
    band = 20000000000
    
    # Compute new cpacity
    embb = int(prediction[0]*band)
    urllc = int(prediction[1]*band)
    miot = int(prediction[2]*band)
    mmtc = int(prediction[3]*band)
    voice = int(prediction[4]*band)

    out = { 'eMBB': embb,
            'URLLC': urllc,
            'MIoT': miot,
            'mMTC': mmtc,
            'voice': voice
          }
    
    return out

collected, slice_weights = 0, []
for key, value in slices_type.items():
    collected += value['client_weight']
    slice_weights.append(collected)

collected, mb_weights = 0, []
for key, value in mobility_patterns.items():
    collected += value['client_weight']
    mb_weights.append(collected)
    
m_patterns = []
for name, mb in mobility_patterns.items():
    mobility_pattern = Mobility(name, mb['distribution'], mb['params'])
    m_patterns.append(mobility_pattern)   

# Load RNN
model = tf.keras.models.load_model('model.h5')
    

In [33]:
# Define class Request
class Request:
    def __init__(self, id_client, type_slice, traffic_request, traffic_left):
        self.id_client = id_client
        self.type_slice = type_slice
        self.traffic_request = traffic_request
        self.traffic_left = traffic_left
        
#Define class csv
class CsvData:
    def __init__(self, step, r_embb, g_embb, connect_embb, r_urllc, g_urllc, connect_urllc, r_miot, g_miot, connect_miot, r_mmtc, g_mmtc, connect_mmtc, r_voice, g_voice, connect_voice):
        self.step = step
        self.r_embb = r_embb
        self.g_embb = g_embb
        self.connect_embb = connect_embb
        self.r_urllc = r_urllc
        self.g_urllc = g_urllc
        self.connect_urllc = connect_urllc
        self.r_miot = r_miot
        self.g_miot = g_miot
        self.connect_miot = connect_miot
        self.r_mmtc = r_mmtc
        self.g_mmtc = g_mmtc
        self.connect_mmtc = connect_mmtc
        self.r_voice = r_voice
        self.g_voice = g_voice
        self.connect_voice = connect_voice
        

In [34]:
# Round 0: create network area, place base station and clients
num_clients = 2000
simulation_time = 2000
x_area = 600
y_area = 600

bs = BaseStation(0, basestation.get('x'), basestation.get('y'), basestation.get('coverage'), basestation.get('capacity'), basestation.get('ratios'))

clients = []
for i in range(num_clients):
    x = r.randint(0, x_area)
    y = r.randint(0, y_area)
    freq = r.random()
    
    mobility_pattern = get_random_mobility_pattern(mb_weights, m_patterns)
    slice_type = get_random_slice_type(slice_weights) 
    
    c = Client(i, x, y, mobility_pattern, freq, slice_type, bs)
    clients.append(c)


In [35]:
active_clients = []
request_list = [None] * len(clients) # create fixed size list to manage the request
csv_final = []
rem, rur, rmi, rmm, rvo = 0, 0, 0, 0, 0

for n in range(simulation_time):
    
    # take only client in range
    clients_inRange = []
    for item in clients:
        if inRange(item, bs):
            clients_inRange.append(item)
    
    # manage number active clients
    active = []
    new_connection = r.randint(0, len(clients_inRange))
    active = r.sample(clients_inRange, new_connection)
    
    for item in active:
        if item.connected == False:
            active_clients.append(item)
    
    # preditc ratios
    new_ratios = [[[rem, rur, rmi, rmm, rvo]]]
    new_ratios = np.array(new_ratios)/bs.capacity
    prediction = model.predict(new_ratios)
    capacity = get_new_ratios(prediction[0])
    
    print('STEP', n)
    
    for item in active_clients:   
        # step 1 and create request
        item.step_1()
        
        req = Request(item.ID, item.slice_type,item.usage_rem, item.usage_rem)
        request_list[item.ID] = req
        
    #user_embb, user_urllc, user_miot, user_mmtc, user_voice = 0, 0, 0, 0, 0
    req_embb, req_urllc, req_miot, req_mmtc, req_voice = 0, 0, 0, 0, 0
    #giv_embb, giv_urllc, giv_miot, giv_mmtc, giv_voice = 0, 0, 0, 0, 0
    for item in request_list:
     if item is not None:
        if item.type_slice == 'eMBB':
            req_embb += item.traffic_request
            #giv_embb += item.traffic_left 
            #user_embb += 1
                
        if item.type_slice == 'URLLC' :
            req_urllc += item.traffic_request
            #giv_urllc += item.traffic_left
            #user_urllc += 1
 
        if item.type_slice == 'MIoT':
            req_miot += item.traffic_request
            #giv_miot += item.traffic_left
            #user_miot += 1

        if item.type_slice == 'mMTC':
            req_mmtc += item.traffic_request
            #giv_mmtc += item.traffic_left
            #user_mmtc += 1

        if item.type_slice == 'voice':
            req_voice += item.traffic_request
            #giv_voice += item.traffic_left
            #user_voice += 1
            
    #DELETE ZERO VALUES
    if bs.diocane != 0 and bs.urllc_active !=0 and bs.dioboia != 0 and bs.miot_active != 0 and bs.boiadio != 0 and bs.mmtc_active !=0:
        csv_data = CsvData(n, bs.uno, bs.porcodio, bs.embb_active, bs.due, bs.diocane, bs.urllc_active, bs.tre, bs.dioboia, bs.miot_active, bs.quattro, bs.boiadio, bs.mmtc_active, bs.cinque, bs.diobestia, bs.voice_active)  
        csv_final.append(csv_data)

  
    for item in active_clients:
        # step 2 and update request (delete non active user, and update traffic)
        item.step_2()
        
    for item in clients:
        # step 3 move all clients
        item.step_3()
    
    tmp = []
    for item in active_clients:    
        if item.connected == True and not inRange(item, bs):
            item.disconnect()
        if item.connected == True:
            request_list[item.ID] = None
            tmp.append(item)
    
    active_clients = tmp
    
    active_clients = tmp
    bs.embb_active = 0
    bs.urllc_active = 0
    bs.miot_active = 0
    bs.mmtc_active = 0
    bs.voice_active = 0
    bs.porcodio = 0
    bs.diocane = 0
    bs.dioboia = 0
    bs.boiadio = 0
    bs.diobestia = 0
    bs.uno = 0
    bs.due = 0
    bs.tre = 0
    bs.quattro = 0
    bs.cinque = 0

            

STEP 0
STEP 1
STEP 2
STEP 3
STEP 4
STEP 5
STEP 6
STEP 7
STEP 8
STEP 9
STEP 10
STEP 11
STEP 12
STEP 13
STEP 14
STEP 15
STEP 16
STEP 17
STEP 18
STEP 19
STEP 20
STEP 21
STEP 22
STEP 23
STEP 24
STEP 25
STEP 26
STEP 27
STEP 28
STEP 29
STEP 30
STEP 31
STEP 32
STEP 33
STEP 34
STEP 35
STEP 36
STEP 37
STEP 38
STEP 39
STEP 40
STEP 41
STEP 42
STEP 43
STEP 44
STEP 45
STEP 46
STEP 47
STEP 48
STEP 49
STEP 50
STEP 51
STEP 52
STEP 53
STEP 54
STEP 55
STEP 56
STEP 57
STEP 58
STEP 59
STEP 60
STEP 61
STEP 62
STEP 63
STEP 64
STEP 65
STEP 66
STEP 67
STEP 68
STEP 69
STEP 70
STEP 71
STEP 72
STEP 73
STEP 74
STEP 75
STEP 76
STEP 77
STEP 78
STEP 79
STEP 80
STEP 81
STEP 82
STEP 83
STEP 84
STEP 85
STEP 86
STEP 87
STEP 88
STEP 89
STEP 90
STEP 91
STEP 92
STEP 93
STEP 94
STEP 95
STEP 96
STEP 97
STEP 98
STEP 99
STEP 100
STEP 101
STEP 102
STEP 103
STEP 104
STEP 105
STEP 106
STEP 107
STEP 108
STEP 109
STEP 110
STEP 111
STEP 112
STEP 113
STEP 114
STEP 115
STEP 116
STEP 117
STEP 118
STEP 119
STEP 120
STEP 121
STEP 122
STE

STEP 923
STEP 924
STEP 925
STEP 926
STEP 927
STEP 928
STEP 929
STEP 930
STEP 931
STEP 932
STEP 933
STEP 934
STEP 935
STEP 936
STEP 937
STEP 938
STEP 939
STEP 940
STEP 941
STEP 942
STEP 943
STEP 944
STEP 945
STEP 946
STEP 947
STEP 948
STEP 949
STEP 950
STEP 951
STEP 952
STEP 953
STEP 954
STEP 955
STEP 956
STEP 957
STEP 958
STEP 959
STEP 960
STEP 961
STEP 962
STEP 963
STEP 964
STEP 965
STEP 966
STEP 967
STEP 968
STEP 969
STEP 970
STEP 971
STEP 972
STEP 973
STEP 974
STEP 975
STEP 976
STEP 977
STEP 978
STEP 979
STEP 980
STEP 981
STEP 982
STEP 983
STEP 984
STEP 985
STEP 986
STEP 987
STEP 988
STEP 989
STEP 990
STEP 991
STEP 992
STEP 993
STEP 994
STEP 995
STEP 996
STEP 997
STEP 998
STEP 999
STEP 1000
STEP 1001
STEP 1002
STEP 1003
STEP 1004
STEP 1005
STEP 1006
STEP 1007
STEP 1008
STEP 1009
STEP 1010
STEP 1011
STEP 1012
STEP 1013
STEP 1014
STEP 1015
STEP 1016
STEP 1017
STEP 1018
STEP 1019
STEP 1020
STEP 1021
STEP 1022
STEP 1023
STEP 1024
STEP 1025
STEP 1026
STEP 1027
STEP 1028
STEP 1029
STEP 10

STEP 1754
STEP 1755
STEP 1756
STEP 1757
STEP 1758
STEP 1759
STEP 1760
STEP 1761
STEP 1762
STEP 1763
STEP 1764
STEP 1765
STEP 1766
STEP 1767
STEP 1768
STEP 1769
STEP 1770
STEP 1771
STEP 1772
STEP 1773
STEP 1774
STEP 1775
STEP 1776
STEP 1777
STEP 1778
STEP 1779
STEP 1780
STEP 1781
STEP 1782
STEP 1783
STEP 1784
STEP 1785
STEP 1786
STEP 1787
STEP 1788
STEP 1789
STEP 1790
STEP 1791
STEP 1792
STEP 1793
STEP 1794
STEP 1795
STEP 1796
STEP 1797
STEP 1798
STEP 1799
STEP 1800
STEP 1801
STEP 1802
STEP 1803
STEP 1804
STEP 1805
STEP 1806
STEP 1807
STEP 1808
STEP 1809
STEP 1810
STEP 1811
STEP 1812
STEP 1813
STEP 1814
STEP 1815
STEP 1816
STEP 1817
STEP 1818
STEP 1819
STEP 1820
STEP 1821
STEP 1822
STEP 1823
STEP 1824
STEP 1825
STEP 1826
STEP 1827
STEP 1828
STEP 1829
STEP 1830
STEP 1831
STEP 1832
STEP 1833
STEP 1834
STEP 1835
STEP 1836
STEP 1837
STEP 1838
STEP 1839
STEP 1840
STEP 1841
STEP 1842
STEP 1843
STEP 1844
STEP 1845
STEP 1846
STEP 1847
STEP 1848
STEP 1849
STEP 1850
STEP 1851
STEP 1852
STEP 1853


In [36]:
with open('new_data_2000.csv', mode='w') as csv_file:
    fieldnames = ['embb_rate', 'embb_users', 'embb_req', 'urllc_rate', 'urllc_users', 'urllc_req', 'miot_rate', 'miot_users', 'miot_req', 'mmtc_rate', 'mmtc_users', 'mmtc_req', 'voice_rate', 'voice_users', 'voice_req']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    
    writer.writeheader()
    for item in csv_final:
        writer.writerow({'embb_rate': item.g_embb, 'embb_users': item.connect_embb, 'embb_req': item.r_embb, 'urllc_rate': item.g_urllc, 'urllc_users': item.connect_urllc, 'urllc_req': item.r_urllc, 'miot_rate': item.g_miot, 'miot_users': item.connect_miot, 'miot_req': item.r_miot, 'mmtc_rate': item.g_mmtc, 'mmtc_users': item.connect_mmtc, 'mmtc_req': item.r_mmtc, 'voice_rate': item.g_voice, 'voice_users': item.connect_voice, 'voice_req': item.r_voice})
